In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pandas as pd # Pandas CSV loading
import numpy as np # Numpy for loading calculations into multidimensional arrays

# Data visualization
import seaborn as sns
import matplotlib as m
import matplotlib as mpl
import matplotlib.pyplot as plt
import plotly
import plotly.express as px

# Warnings remove alerts
import warnings
warnings.filterwarnings("ignore")

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [6]:
# NLTK to NLP
import nltk
import re

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nltk.download('vader_lexicon')
nltk.download("stopwords")
nltk.download('punkt')

In [8]:
# Loading database
df=pd.read_csv('/kaggle/input/twitter-airline-sentiment/Tweets.csv')

In [9]:
# Displayed first 5 data
df.head()

In [10]:
# Displayed last 5 data 
df.tail(5)

In [11]:
# Number of rows and columns
df.shape

In [12]:
# Checking variable information
df.info()

In [13]:
# Data type
df.dtypes

In [14]:
# Total columns and rows 

print("Números de linhas: {}" .format(df.shape[0]))
print("Números de colunas: {}" .format(df.shape[1]))

In [15]:
# Displaying missing values and unique values

print("\nMissing values :  ", df.isnull().sum().values.sum())
print("\nUnique values :  \n",df.nunique())

# **Data Visualization**

In [16]:
import warnings   
from wordcloud import WordCloud
from termcolor import cprint

In [17]:
plt.figure(figsize=(24,20))

ax = sns.countplot(x = 'airline_sentiment', data = df)
plt.title("Sentiment Analysis - Airlines")
plt.xlabel("Feelings")
plt.ylabel("Total")

In [18]:
plt.figure(figsize=(14.8, 6))

ax = sns.boxplot(x="airline", y="airline_sentiment_confidence", data = df)

In [19]:
plt.figure(figsize=(14.8, 6))

ax = sns.boxplot(x="airline_sentiment", y="airline_sentiment_confidence", data = df)

In [20]:
colors=sns.color_palette('husl',10)
pd.Series(df['airline']).value_counts().plot(kind="bar",color=colors,figsize=(10,8),fontsize=10,rot=0,title='Airlines total feelings')
plt.xlabel("Feelings", fontsize=10)
plt.ylabel("Total", fontsize=10)

In [21]:
plt.figure(figsize=(24,20))

ax = sns.countplot(x = 'airline', data = df)
plt.title("Sentiment Analysis - Airlines")
plt.xlabel("Feelings")
plt.ylabel("Total")

# Word cloud

**Positive feelings word cloud**

In [22]:
positive=df[df['airline_sentiment']=='positive'].text
neutral=df[df['airline_sentiment']=='neutral'].text
negative=df[df['airline_sentiment']=='negative'].text

plt.figure(figsize=(24,20))

world_cloud_postive=WordCloud(min_font_size=3,max_words=3200,width=1600,height=720).generate("".join(positive))
plt.imshow(world_cloud_postive,interpolation='bilinear')
ax.grid(False)

**Neutral feelings word cloud**

In [23]:
plt.figure(figsize=(24,12))

world_cloud_neutral=WordCloud(min_font_size=3,max_words=3200,width=1600,height=720).generate(" ".join(neutral))
plt.imshow(world_cloud_neutral,interpolation='bilinear')
ax.grid(False)

**Negative feelings word cloud**

In [24]:
plt.figure(figsize = (24,12))

worldcould_neg = WordCloud(min_font_size = 3,  max_words = 3200 , width = 1600 , height = 720).generate(" ".join(negative))
plt.imshow(worldcould_neg,interpolation = 'bilinear')
ax.grid(False)

# Pre-processing

In [25]:
# Creating a function to bring in number 0, 1, 2
def sentimento(sentimento):
    if sentimento == "positive":
        return 2
    elif sentimento == "neutral":
        return 1
    elif sentimento == "negative":
        return 0

# Applying the feeling function
df.airline_sentiment = df.airline_sentiment.apply(lambda x: sentimento(x))
print("Sentimentos:", df.airline_sentiment)

In [26]:
# Checking review
df.airline_sentiment.count()

In [27]:
# Checking base
df.value_counts()

# Training and Testing

- Review column database training and testing, sentiment

In [28]:
# Variable for testing
treino = df["text"]

# Variable for training
teste = df["airline_sentiment"]

In [29]:
# Total rows and columns given variable x
treino.shape

In [30]:
# Total rows and columns given variable y
teste.shape

In [31]:
# Cleanup data for PLN model

import re
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

# Removing stop words
def remove_stop_words(instancia):
    stopwords = set(nltk.corpus.stopwords.words("english"))
    palavras = [i for i in instancia.split() if not i in stopwords]
    return (" ".join(palavras))

# stemming no dataset
def text_stemming(instancia):
    stemmer = nltk.stem.RSLPStemmer()

    palavras = []

    for w in instancia.split():
        palavras.append(stemmer.stem(w))
    return (" ".join(palavras))

# Cleaning database
def dados_limpos(instancia):
    instancia = re.sub(r"http\S+", "", instancia).lower().replace('.','').replace(';','').replace('-','').replace(':','').replace(')','')
    return (instancia)

# Reduction of inflected words
def Lemmatization(instancia):
    
    palavras = []
    
    for w in instancia.split():
        palavras.append(wordnet_lemmatizer.lemmatize(w))
        return (" ".join(palavras))

# Preprocessing removing stopwords removing unwanted characters.
def Preprocessing(instancia):
    instancia = re.sub(r"http\S+", "", instancia).lower().replace('.','').replace(';','').replace('-','').replace(':','').replace(')','').replace('"','')
    stopwords = set(nltk.corpus.stopwords.words('portuguese'))
    palavras = [i for i in instancia.split() if not i in stopwords]
    return (" ".join(palavras))

# Viewing text
treino = [Preprocessing(i) for i in treino]
treino[:20]

# TFIDF - tweets

In [32]:
# Word tokenizer - And the process of splitting a string, texts and a list of tokens 
# Template created to sort positive, negative tweets

from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import TweetTokenizer

model_tf = TweetTokenizer()
model_tf_vect = CountVectorizer(analyzer="word", tokenizer = model_tf.tokenize)
model_tf = model_tf_vect.fit(treino)

In [33]:
# Tweet Tokenizer pre-processing
from nltk.tokenize import TweetTokenizer
from nltk.tokenize import word_tokenize

text = "Hoje passei no concurso eng da computação, mais teve muita gente para concurso"
word_tokenize(text)


tokenizer = TweetTokenizer()
tokenizer.tokenize(text)

# Count Vectorizer do modelo

In [34]:
# CountVectorizer create a vocabulary of words and return in vector
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(analyzer="word", tokenizer = tokenizer.tokenize)
freq = vectorizer.fit_transform(treino)
freq.shape

# Model - Pipeline 

In [35]:
from sklearn.model_selection import cross_val_predict
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

pipeline_NB = Pipeline([  # Pipeline 1
    ("counts", CountVectorizer()), # CountVectorizer training
    ("classifier", MultinomialNB()) # Model naive bayes classification
])
pipeline_NB

In [36]:
# Forecast 
result_pipeline_NLP = cross_val_predict(pipeline_NB, treino, teste, cv = 10)
result_pipeline_NLP

In [37]:
# Accuracy
from sklearn.metrics import accuracy_score

accuracy_naive_bayes_multinomialNB = accuracy_score(teste, result_pipeline_NLP)
print("Accuracy - Pipeline Naive bayes multinomialNB: %.2f" % (accuracy_naive_bayes_multinomialNB * 100))

In [38]:
# Classification report
from sklearn.metrics import classification_report

classification = classification_report(teste, result_pipeline_NLP)
print("Pipeline Naive bayes")
print()
print(classification)

In [39]:
# Confusion matrix
from sklearn.metrics import confusion_matrix

matrix = confusion_matrix(result_pipeline_NLP, teste)

x = ["negative", "neutral", "positive"]
y = ['negative', "neutral", "positive"]

matrix = pd.DataFrame(matrix, columns=np.unique(y), index = np.unique(x))
matrix.index.name = 'Actual'
matrix.columns.name = 'Predicted'

plt.figure(figsize = (10,7))
sns.set(font_scale=1.4)
plt.title("Matrix confusion")
matrix = sns.heatmap(matrix, cmap = 'plasma', annot=True, annot_kws = {"size": 20}, fmt = "")

# Pipeline 2 - XGBosst Classifier

In [40]:
# Pipeline XGBosst Classifier
from xgboost import XGBClassifier

pipeline_2 = Pipeline([
    ("counts", CountVectorizer()),
    ("classifier", XGBClassifier())
])
pipeline_2

In [41]:
# Forecast from the XGBoost pipeline model
result_pipeline_2 = cross_val_predict(pipeline_2, treino, teste, cv = 10)
result_pipeline_2

In [42]:
# Accuracy score model
pipeline_XGBosst = accuracy_score(teste, result_pipeline_2)
print("Accuracy - XGBClassifier: %.2f" % (pipeline_XGBosst * 100))

In [43]:
# Confusion matrix
matrix = confusion_matrix(result_pipeline_2, teste)

x = ["negative", "neutral", "positive"]
y = ['negative', "neutral", "positive"]

matrix = pd.DataFrame(matrix, columns=np.unique(y), index = np.unique(x))
matrix.index.name = 'Actual'
matrix.columns.name = 'Predicted'

plt.figure(figsize = (10,7))
sns.set(font_scale=1.4)
plt.title("Matrix confusion - XGBClassifier")
matrix = sns.heatmap(matrix, cmap = 'plasma', annot=True, annot_kws = {"size": 20}, fmt = "")

In [44]:
# Results - Machine learning models

modelos = pd.DataFrame({
    
    "Models" :["Pipeline 1: Naive bayes", 
               "Pipeline 2: XGBosst"],

    "Acurácia" :[accuracy_naive_bayes_multinomialNB,
                 pipeline_XGBosst]})

modelos_2 = modelos.sort_values(by = "Acurácia", ascending = True)
modelos_2.to_csv("modelos_2.csv")
modelos_2

In [45]:
# Saving Machine learning pipeline

import pickle    
    
with open('result_pipeline_NLP.pkl', 'wb') as file:
    pickle.dump(result_pipeline_NLP, file)
    
with open('result_pipeline_2.pkl', 'wb') as file:
    pickle.dump(result_pipeline_2, file)